<a href="https://colab.research.google.com/github/KillM0nGerZ/WangchanBERTa-for-QuestionAnswering/blob/main/WangchanBERTa_for_QuestionAnswering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers==4.11.2
!pip install sentencepiece

     |████████████████████████████████| 325 kB 4.3 MB/s 
     |████████████████████████████████| 2.9 MB 56.3 MB/s 
     |████████████████████████████████| 67 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 52.5 MB/s 
     |████████████████████████████████| 895 kB 49.9 MB/s 
     |████████████████████████████████| 3.3 MB 43.4 MB/s 
     |████████████████████████████████| 1.1 MB 56.5 MB/s 
     |████████████████████████████████| 212 kB 51.7 MB/s 
     |████████████████████████████████| 134 kB 63.2 MB/s 
     |████████████████████████████████| 127 kB 67.1 MB/s 
     |████████████████████████████████| 271 kB 56.8 MB/s 
     |████████████████████████████████| 144 kB 57.1 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

#Preprocess data

In [2]:
squad_v2 = False
model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
batch_size = 16

In [3]:
from datasets import load_dataset, load_metric

In [4]:
datasets = load_dataset("thaiqa_squad")

Generating train split:   0%|          | 0/4000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/74 [00:00<?, ? examples/s]

Dataset thaiqa_squad downloaded and prepared to /root/.cache/huggingface/datasets/thaiqa_squad/thaiqa_squad/1.0.0/fce14864b511d48464540780f328f4b415746b63f2fd934ad0b06c3eead7787b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['question_id', 'article_id', 'context', 'question', 'answers'],
        num_rows: 74
    })
})

In [6]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/883k [00:00<?, ?B/s]

In [7]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [8]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [9]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [10]:
len(tokenizer(example["question"], example["context"])["input_ids"])

520

In [11]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [12]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [13]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 294]

In [14]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

<s> สุนัขตัวแรกรับบทเป็นเบนจี้ในภาพยนตร์เรื่อง <unk>enji ที่ออกฉายในปี พ.ศ. 2517 มีชื่อว่าอะไร</s></s> <doc id="115035" url="https://th.wikipedia.org/wiki?curid=115035" title="เบนจี้">เบนจี้ เบนจี้ () เป็นชื่อตัวละครหมาพันทางแสนรู้ ที่ปรากฏอยู่ในภาพยนตร์หลายเรื่องที่เขียนบท และกํากับโดย โจ แคมป์ ในช่วงทศวรรษ 1970 ถึง 1980 ภาพยนตร์เรื่องแรกในชุด ใช้ชื่อเรื่องว่า เบนจี้ เช่นเดียวกับตัวละคร ถ่ายทําที่เมืองดัลลัส รัฐเทกซัส ฉายครั้งแรกในปี พ.ศ. 2517 ภาพยนตร์ได้รับการเสนอชื่อเข้าชิงรางวัลออสการ์ และได้รางวัลลูกโลกทองคํา สาขาเพลงประกอบยอดเยี่ยม จากเพลง <unk>enji's <unk>heme (<unk> <unk>eel <unk>ove) ร้องโดย ชาร์ลี ริช หมาที่แสดงเป็นเบนจี้ตัวแรก ชื่อว่า ฮิกกิ้นส์ (พ.ศ. 2502 - พ.ศ. 2518) มีอายุถึง 15 ปีแล้วในขณะแสดง หลังจากภาพยนตร์ออกฉายได้ไม่นาน มันก็ตายในปี พ.ศ. 2518เบนจี้ในภาพยนตร์เบนจี้ในภาพยนตร์. - พ.ศ. 2517, <unk>enji (ภาพยนตร์) - พ.ศ. 2520, <unk>or the <unk>ove of <unk>enji (ภาพยนตร์) - พ.ศ. 2521, <unk>enji's <unk>ery <unk>wn <unk>hristmas <unk>tory (ภาพยนตร์โทรทัศน์) - พ.ศ. 2523, <unk>h

In [15]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 1), (0, 5), (5, 11), (11, 20), (20, 23), (23, 26), (26, 36), (36, 42), (42, 43), (43, 44), (44, 46), (46, 48), (48, 49), (49, 52), (52, 58), (58, 62), (62, 63), (63, 64), (64, 65), (65, 66), (66, 67), (67, 68), (68, 72), (72, 73), (73, 82), (82, 86), (0, 0), (0, 0), (0, 1), (0, 1), (1, 3), (3, 4), (4, 5), (5, 7), (7, 8), (8, 9), (9, 10), (10, 13), (13, 15), (15, 16), (16, 17), (17, 19), (19, 20), (20, 21), (21, 22), (22, 27), (27, 30), (30, 32), (32, 33), (33, 42), (42, 43), (43, 46), (46, 47), (47, 51), (51, 52), (52, 55), (55, 57), (57, 59), (59, 62), (62, 64), (64, 65), (65, 66), (66, 71), (71, 72), (72, 73), (73, 76), (76, 79), (79, 80), (80, 81), (81, 84), (84, 87), (87, 88), (88, 91), (91, 94), (94, 96), (96, 97), (97, 98), (98, 106), (106, 113), (113, 116), (116, 119), (119, 122), (122, 125), (125, 128), (128, 129), (129, 132), (132, 141), (141, 151), (151, 161), (161, 164), (164, 171), (171, 175), (175, 183), (183, 184), (184, 186), (186, 187), (187, 192), (192, 19

In [16]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

▁ ส


In [17]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [18]:
answers = (example["answers"])
start_char = answers["answer_begin_position"][0]
end_char = start_char + len(answers["answer"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

198 201


In [19]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["answer"][0])

ฮิกกิ้นส์
ฮิกกิ้นส์


In [20]:
pad_on_right = tokenizer.padding_side == "right"

In [21]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = (examples["answers"][sample_index])
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_begin_position"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_begin_position"][0]
            end_char = start_char + len(answers["answer"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [22]:
features = prepare_train_features(datasets['train'][:5])

In [23]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

#Fine-tune Model

In [24]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/404M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForQuestionAnswering: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForQuestionAnswering were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['qa_outputs.bias', 

In [25]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    save_steps=2000,
)

In [26]:
from transformers import default_data_collator

data_collator = default_data_collator

In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 20118
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1258


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.save_model("./model-QA-wangchanberta")

In [ ]:
tokenizer.save_pretrained("./model-QA-wangchanberta")

#Prediction

In [ ]:
from transformers import pipeline
qa_model = pipeline(model="/content/model-QA-wangchanberta", 
                    tokenizer="/content/model-QA-wangchanberta", 
                    task="question-answering")

In [31]:
que = 'ดึกแล้วคุณขา เป็นภาพยนตร์แนวใด'
text ='''ดึกแล้วคุณขา ดึกแล้วคุณขา (Bangkok Time) เป็นภาพยนตร์ไทยที่กำกับโดยสันติ แต้พานิช ออกฉายในปี พ.ศ. 2550 
        โดยฉายแบบจำกัดโรงที่โรงภาพยนตร์ลิโด้ ดึกแล้วคุณขา เป็นภาพยนตร์รัก โรแมนติก มีเนื้อหาเกี่ยวกับรักสามเส้า ของชายสองหญิงหนึ่ง
        ที่เป็นคนทำงานกลางคืน คือ พ่อค้าขายของปลอมริมถนนสีลม (รับบทโดย อรรถพร ธีมากร) ชายหนุ่มบริการทางโทรศัพท์ (รับบทโดย อนันดา เอเวอริ่งแฮม) 
        และนางพยาบาลที่ต้องเข้าเวรกะดึก (รับบทโดย ดุสิตา อนุชิตชาญชัย) ภาพยนตร์เรื่องนี้เข้าฉายใน Bangkok Film Festival 2007 และ 
        Vancouver International Film Festival 2007 ในสาขา Dragon and Tiger Award'''
qa_model(question=que, context = text)

{'answer': ' เป็นภาพยนตร์รัก โรแมนติก',
 'end': 186,
 'score': 0.0006689970032311976,
 'start': 161}